In [ ]:
import os
import shutil
import random
import warnings
import zipfile
import os
import shutil
import numpy as np
import pandas as pd
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, BatchNormalization, Dropout
from tensorflow.keras.regularizers import L2
from tensorflow.keras.applications import ResNet50q
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.regularizers import l2
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, BatchNormalization, Dropout
from tensorflow.keras.regularizers import L2
from google.colab import files
from sklearn.model_selection import train_test_split

warnings.filterwarnings('ignore')

In [ ]:


uploaded = files.upload()


zip_filename = list(uploaded.keys())[0]


with zipfile.ZipFile(zip_filename, 'r') as zip_ref:
    zip_ref.extractall('/content/garbage_dataset')

print("Extraction complete! Files are in /content/garbage_dataset")


Saving archive.zip to archive.zip
Extraction complete! Files are in /content/garbage_dataset


In [ ]:



dataset_dir = "/content/garbage_dataset/garbage_classification/Garbage_classification"

total_images = 0
class_counts = {}

for class_name in os.listdir(dataset_dir):
    class_path = os.path.join(dataset_dir, class_name)
    if os.path.isdir(class_path):
        count = len([f for f in os.listdir(class_path) if f.lower().endswith(('.png', '.jpg', '.jpeg'))])
        class_counts[class_name] = count
        total_images += count


for cls, count in class_counts.items():
    print(f"{cls}: {count}")

print("\nTotal images:", total_images)

cardboard: 403
paper: 594
plastic: 482
glass: 501
trash: 137
metal: 410

Total images: 2527


In [ ]:



data_dir = "/content/garbage_dataset/garbage_classification/Garbage_classification"
output_dir = "/content"


for split in ["train", "val", "test"]:
    os.makedirs(os.path.join(output_dir, split), exist_ok=True)


test_size = 0.1
val_size = 0.1

# Iterate over each class folder
for class_name in os.listdir(data_dir):
    class_path = os.path.join(data_dir, class_name)
    if not os.path.isdir(class_path):
        continue

    # Get all images in class
    images = [os.path.join(class_path, img) for img in os.listdir(class_path) if img.lower().endswith(('.png', '.jpg', '.jpeg'))]

    # Train/test split
    train_imgs, test_imgs = train_test_split(images, test_size=test_size, random_state=42)
    train_imgs, val_imgs = train_test_split(train_imgs, test_size=val_size/(1 - test_size), random_state=42)

    # Function to copy
    def copy_images(img_list, split):
        split_dir = os.path.join(output_dir, split, class_name)
        os.makedirs(split_dir, exist_ok=True)
        for img in img_list:
            shutil.copy(img, split_dir)

        print(" cpy done ")

    # Copy files
    copy_images(train_imgs, "train")
    copy_images(val_imgs, "val")
    copy_images(test_imgs, "test")


print(" Dataset split into train/val/test successfully.")


✅ Dataset split into train/val/test successfully.


In [ ]:



train_dir = "/content/train"
val_dir   = "/content/val"
test_dir  = "/content/test"


train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True
)

val_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

train_gen = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

val_gen = val_datagen.flow_from_directory(
    val_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

test_gen = test_datagen.flow_from_directory(
    test_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    shuffle=False
)


base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))






early_stopper = EarlyStopping(monitor='val_loss', patience=5, verbose=1, mode='min', restore_best_weights=True)



model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(256, activation='relu', kernel_regularizer=l2(0.001)),
    BatchNormalization(),
    Dropout(0.5),
    Dense(6, activation='softmax')
])
base_model.trainable = True
for layer in base_model.layers[:-45]:
    layer.trainable = False



model.compile(optimizer=Adam(learning_rate=1e-3),
              loss='categorical_crossentropy',
              metrics=['accuracy'])


history = model.fit(
    train_gen,
    epochs=15,
    validation_data=val_gen,
    callbacks=[early_stopper]
)




test_loss, test_acc = model.evaluate(test_gen)
print(f"Test accuracy: {test_acc:.2f}")

Found 2016 images belonging to 6 classes.
Found 255 images belonging to 6 classes.
Found 256 images belonging to 6 classes.
Epoch 1/15
63/63 ━━━━━━━━━━━━━━━━━━━━ 615s 9s/step - accuracy: 0.6222 - loss: 1.7824 - val_accuracy: 0.1725 - val_loss: 113.6736
Epoch 2/15
63/63 ━━━━━━━━━━━━━━━━━━━━ 609s 9s/step - accuracy: 0.8693 - loss: 0.8431 - val_accuracy: 0.7020 - val_loss: 2.4543
Epoch 3/15
63/63 ━━━━━━━━━━━━━━━━━━━━ 623s 9s/step - accuracy: 0.9315 - loss: 0.6289 - val_accuracy: 0.8275 - val_loss: 1.0155
Epoch 4/15
63/63 ━━━━━━━━━━━━━━━━━━━━ 627s 9s/step - accuracy: 0.9303 - loss: 0.6169 - val_accuracy: 0.7765 - val_loss: 1.4508
Epoch 5/15
63/63 ━━━━━━━━━━━━━━━━━━━━ 593s 9s/step - accuracy: 0.9581 - loss: 0.4967 - val_accuracy: 0.8667 - val_loss: 0.8665
Epoch 6/15
63/63 ━━━━━━━━━━━━━━━━━━━━ 585s 9s/step - accuracy: 0.9566 - loss: 0.4803 - val_accuracy: 0.8235 - val_loss: 1.1277
Epoch 7/15
63/63 ━━━━━━━━━━━━━━━━━━━━ 586s 9s/step - accuracy: 0.9600 - loss: 0.4118 - val_accuracy: 0.8039 - va

In [ ]:

model.save('zepala_model.h5')
print("✅ Model saved successfully.")


✅ Model saved successfully.


In [ ]:
model.save_weights('model_weights_22.weights.h5')
print(" Weights saved successfully")


✅ Weights saved successfully


In [ ]:
test_gen.reset()

pred_probs = model.predict(test_gen)
pred_classes = np.argmax(pred_probs, axis=1)  # predicted class indices
true_classes = test_gen.classes               # actual class indices
class_labels = list(test_gen.class_indices.keys())

for i in range(10):
    print(f"Image {i+1}: True = {class_labels[true_classes[i]]}, Predicted = {class_labels[pred_classes[i]]}")

8/8 ━━━━━━━━━━━━━━━━━━━━ 41s 5s/step
Image 1: True = cardboard, Predicted = cardboard
Image 2: True = cardboard, Predicted = cardboard
Image 3: True = cardboard, Predicted = cardboard
Image 4: True = cardboard, Predicted = cardboard
Image 5: True = cardboard, Predicted = paper
Image 6: True = cardboard, Predicted = cardboard
Image 7: True = cardboard, Predicted = cardboard
Image 8: True = cardboard, Predicted = cardboard
Image 9: True = cardboard, Predicted = cardboard
Image 10: True = cardboard, Predicted = cardboard


In [ ]:
from google.colab import files
import shutil


shutil.make_archive('/content/train_folder', 'zip', '/content/train')
shutil.make_archive('/content/val_folder', 'zip', '/content/val')
shutil.make_archive('/content/test_folder', 'zip', '/content/test')


files.download('/content/train_folder.zip')
files.download('/content/val_folder.zip')
files.download('/content/test_folder.zip')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ resnet50 (Functional)           │ (None, 7, 7, 2048)     │    23,587,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_2      │ (None, 2048)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 256)            │       524,544 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 256)            │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 6)              │         1,542 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 57,361,556 (218.82 MB)

 Trainable params: 16,623,366 (63.41 MB)

 Non-trainable params: 7,491,456 (28.58 MB)

 Optimizer params: 33,246,734 (126.83 MB)

## **transfer model**